# Understandable performance
*Going fast, nowhere*

In [13]:
using Pkg
Pkg.activate(;temp=true)
Pkg.add(["BenchmarkTools", "LLVM", "Unitful"])

  Activating new project at `/tmp/jl_skn2MW`
   Resolving package versions...
   Installed Unitful ─ v1.10.1
    Updating `/tmp/jl_skn2MW/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.2.2
  [929cbde3] + LLVM v4.7.1
  [1986cc42] + Unitful v1.10.1
    Updating `/tmp/jl_skn2MW/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.2.2
  [fa961155] + CEnum v0.4.1
  [187b0558] + ConstructionBase v1.3.0
  [692b3bcd] + JLLWrappers v1.4.0
  [682c06a0] + JSON v0.21.2
  [929cbde3] + LLVM v4.7.1
  [69de0a69] + Parsers v2.2.0
  [21216c6a] + Preferences v1.2.3
  [1986cc42] + Unitful v1.10.1
  [dad2f222] + LLVMExtra_jll v0.0.13+1
  [0dad84c5] + ArgTools
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [ade2ca70] + Dates
  [f43a241f] + Downloads
  [b77e0a4c] + InteractiveUtils
  [b27032c2] + LibCURL
  [76f85450] + LibGit2
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [d6f4376e] + Markdown
  [a63ad114] + Mmap
  [ca575930] + NetworkOptions
  [44cfe95a] + Pkg
  [de0858da] + Printf

## A note on benchmarking
*Premature optimization is the root of all evil* & *If you don't measure you won't improve*

### Tools
1. BenchmarkTools.jl https://github.com/JuliaCI/BenchmarkTools.jl
2. Profiler https://docs.julialang.org/en/latest/manual/profile/
3. ProfileView.jl https://github.com/timholy/ProfileView.jl
4. PProf https://github.com/vchuravy/PProf.jl


### Other
1. VTunes/Perf/OProfile https://docs.julialang.org/en/latest/manual/profile/#External-Profiling-1
2. LIKWID 
3. LinuxPerf
4. MCAnalyzer

## BenchmarkTools.jl
Solid package that tries to eliminate common pitfalls in performance measurment.
- `@benchmark` macro that will repeatedly evaluate your code to gain enough samples
- Caveat: You probably want to escape `$` your input data

In [5]:
data = rand(2^10);

In [6]:
function sum(X::Vector{Float64})
    acc = 0::Int64
    for x in X
        (acc += x)::Float64
    end
    acc::Union{Int64, Float64}
end

sum (generic function with 1 method)

In [7]:
using BenchmarkTools
@benchmark sum($data)

BenchmarkTools.Trial: 10000 samples with 123 evaluations.
 Range (min … max):  745.691 ns … 1.104 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     746.504 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   748.123 ns ± 8.416 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▆     ▁▁                                                  ▁
  ████▆▇▆▇███▇▇▆▇▇██▆▆▆▅▆▆▅▅▆▅▆▅▆▇▆▆▆▆▅▅▅▄▅▅▃▅▄▃▄▅▄▄▆▄▄▅▃▄▃▄▄ █
  746 ns       Histogram: log(frequency) by time       781 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

## Figuring out what is happening
The stages of the compiler
- `@code_lowered`
- `@code_typed` & `@code_warntype`
- `@code_llvm`
- `@code_native`

Where is a function defined
`@which` & `@edit`

In [8]:
@code_lowered sum(data)

CodeInfo(
1 ─       acc = Core.typeassert(0, Main.Int64)
│   %2  = X
│         @_3 = Base.iterate(%2)
│   %4  = @_3 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_3
│         x = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│   %10 = acc + x
│         acc = %10
│         Core.typeassert(%10, Main.Float64)
│         @_3 = Base.iterate(%2, %9)
│   %14 = @_3 === nothing
│   %15 = Base.not_int(%14)
└──       goto #4 if not %15
3 ─       goto #2
4 ┄ %18 = acc
│   %19 = Core.apply_type(Main.Union, Main.Int64, Main.Float64)
│   %20 = Core.typeassert(%18, %19)
└──       return %20
)

# A simple example: counting

In [11]:
function f(N)
    acc = 0
    for i in 1:N
        acc += 1
    end
    return acc
end

f (generic function with 1 method)

In [12]:
N = 100_000_000
result = @benchmark f($N)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.249 ns … 26.100 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.260 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.261 ns ±  0.275 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃    ▇    ▃                                              █ ▁
  █▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ █
  1.25 ns      Histogram: log(frequency) by time     1.26 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [15]:
using Unitful

In [16]:
t = time(minimum(result)) * u"ns" # in ns
pFreq = N/t |> u"PHz"
t, pFreq

(1.249 ns, 80.06405124099278 PHz)

So we are doing 100 million additions in 1.2ns.
So our processor is operating at 80 PHz...

We wish...

What is going on?

Let's do a basic check, 10x bigger input

In [17]:
@benchmark f($(10*N))

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.249 ns … 5.190 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.260 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.260 ns ± 0.067 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂    █                                                     
  █▁▁▁▁█▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▂ ▂
  1.25 ns        Histogram: frequency by time       1.37 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Let's explore what code we are **actually** running. 

Using Julia's reflection macros we can see all of the stages of code-generation.

In [18]:
@code_lowered f(N)

CodeInfo(
1 ─       acc = 0
│   %2  = 1:N
│         @_3 = Base.iterate(%2)
│   %4  = @_3 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_3
│         i = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│         acc = acc + 1
│         @_3 = Base.iterate(%2, %9)
│   %12 = @_3 === nothing
│   %13 = Base.not_int(%12)
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return acc
)

In [19]:
@code_typed optimize=false f(N)

CodeInfo(
1 ─       (acc = 0)::Core.Const(0)
│   %2  = (1:N)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))::Union{Nothing, Tuple{Int64, Int64}}
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))::Int64
│   %9  = Core.getfield(%7, 2)::Int64
│         (acc = acc + 1)::Int64
│         (@_3 = Base.iterate(%2, %9))::Union{Nothing, Tuple{Int64, Int64}}
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return acc
) => Int64

In [20]:
@code_typed optimize=true f(N)

CodeInfo(
1 ── %1  = Base.sle_int(1, N)::Bool
│    %2  = Base.ifelse(%1, N, 0)::Int64
│    %3  = Base.slt_int(%2, 1)::Bool
└───       goto #3 if not %3
2 ──       Base.nothing::Nothing
└───       goto #4
3 ──       goto #4
4 ┄─ %8  = φ (#2 => true, #3 => false)::Bool
│    %9  = φ (#3 => 1)::Int64
│    %10 = Base.not_int(%8)::Bool
└───       goto #10 if not %10
5 ┄─ %12 = φ (#4 => %9, #9 => %21)::Int64
│    %13 = φ (#4 => 0, #9 => %14)::Int64
│    %14 = Base.add_int(%13, 1)::Int64
│    %15 = (%12 === %2)::Bool
└───       goto #7 if not %15
6 ──       Base.nothing::Nothing
└───       goto #8
7 ── %19 = Base.add_int(%12, 1)::Int64
└───       goto #8
8 ┄─ %21 = φ (#7 => %19)::Int64
│    %22 = φ (#6 => true, #7 => false)::Bool
│    %23 = Base.not_int(%22)::Bool
└───       goto #10 if not %23
9 ──       goto #5
10 ┄ %26 = φ (#8 => %14, #4 => 0)::Int64
└───       return %26
) => Int64

In [21]:
@code_llvm optimize=false f(10)

;  @ In[11]:1 within `f`
define i64 @julia_f_3683(i64 signext %0) #0 {
top:
  %1 = call {}*** @julia.get_pgcstack()
  %2 = bitcast {}*** %1 to {}**
  %current_task = getelementptr inbounds {}*, {}** %2, i64 2305843009213693940
  %3 = bitcast {}** %current_task to i64*
  %world_age = getelementptr inbounds i64, i64* %3, i64 13
;  @ In[11]:3 within `f`
; ┌ @ range.jl:5 within `Colon`
; │┌ @ range.jl:354 within `UnitRange`
; ││┌ @ range.jl:359 within `unitrange_last`
; │││┌ @ operators.jl:425 within `>=`
; ││││┌ @ int.jl:477 within `<=`
       %4 = icmp sle i64 1, %0
; │││└└
     %5 = zext i1 %4 to i8
     %6 = trunc i8 %5 to i1
     %7 = xor i1 %6, true
     %8 = select i1 %7, i64 0, i64 %0
; └└└
; ┌ @ range.jl:833 within `iterate`
; │┌ @ range.jl:609 within `isempty`
; ││┌ @ operators.jl:378 within `>`
; │││┌ @ int.jl:83 within `<`
      %9 = icmp slt i64 %8, 1
; │└└└
   %10 = zext i1 %9 to i8
   %11 = trunc i8 %10 to i1
   %12 = xor i1 %11, true
   br i1 %12, label %L7, label %L5

L5: 

In [22]:
@code_llvm optimize=true f(10)

;  @ In[11]:1 within `f`
define i64 @julia_f_3706(i64 signext %0) #0 {
top:
;  @ In[11]:3 within `f`
; ┌ @ range.jl:5 within `Colon`
; │┌ @ range.jl:354 within `UnitRange`
; ││┌ @ range.jl:359 within `unitrange_last`
     %.inv = icmp sgt i64 %0, 0
; └└└
  %spec.select = select i1 %.inv, i64 %0, i64 0
;  @ In[11]:6 within `f`
  ret i64 %spec.select
}


In [23]:
@code_native f(10)

	.text
; ┌ @ In[11]:3 within `f`
	movq	%rdi, %rax
	sarq	$63, %rax
	andnq	%rdi, %rax, %rax
; │ @ In[11]:6 within `f`
	retq
	nopl	(%rax)
; └


# Conclusion

LLVM realised that our loop.

```julia
for i in 1:N
  acc += 1
end
```

Just ended up being $acc = 1 * N$

# Exercise

What happens with:

```julia
function g(N)
    acc = 0
    for i in 1:N
        acc += 1.0
    end
    acc
end
```

and
    
```julia
function h(N)
    acc = 0.0
    for i in 1:N
        acc += 1.0
    end
    acc
end
```

Take some time to study the different stages of the compilation pipeline.
    

In [24]:
function g(N)
    acc = 0
    for i in 1:N
        acc += 1.0
    end
    acc
end

g (generic function with 1 method)

In [26]:
function h(N)
    acc = 0.0
    for i in 1:N
        acc += 1.0
    end
    acc
end

h (generic function with 1 method)

In [28]:
function g(::Type{T}, N) where T<:Number
    acc = zero(T)
    for i in 1:N
        acc += one(T)
    end
    acc
end

g (generic function with 2 methods)

# Can we actually measure the speed of our original code?

In [10]:
##########################
# Low-level benchmarking #
##########################
using LLVM
using LLVM.Interop

 """
    clobber()

Force the compiler to flush pending writes to global memory.
Acts as an effective read/write barrier.
"""
@inline clobber() = @asmcall("", "~{memory}", true) 

"""
    escape(val)

The `escape` function can be used to prevent a value or
expression from being optimized away by the compiler. This function is
intended to add little to no overhead.
See: https://youtu.be/nXaxk27zwlk?t=2441
"""
@inline escape(val::T) where T = @asmcall("", "X,~{memory}", true, Nothing, Tuple{T}, val)

┌ Info: Precompiling LLVM [929cbde3-209d-540e-8aea-75f648917ca0]
└ @ Base loading.jl:1423


escape

In [44]:
function k(::Type{T}, N) where T
    acc = zero(T)
    for i in 1:N
        acc += one(T)
        clobber()
    end
    return acc
end

k (generic function with 1 method)

In [52]:
@code_llvm debuginfo=:none k(Int64, 10)

define i64 @julia_k_4629(i64 signext %0) #0 {
top:
  %.inv = icmp sgt i64 %0, 0
  %1 = select i1 %.inv, i64 %0, i64 0
  br i1 %.inv, label %L12, label %L29

L12:                                              ; preds = %L12, %top
  %value_phi2 = phi i64 [ %2, %L12 ], [ 1, %top ]
  call void asm sideeffect "", "~{memory}"() #2
  %.not = icmp eq i64 %value_phi2, %1
  %2 = add nuw i64 %value_phi2, 1
  br i1 %.not, label %L29, label %L12

L29:                                              ; preds = %L12, %top
  %value_phi6 = phi i64 [ 0, %top ], [ %1, %L12 ]
  ret i64 %value_phi6
}


In [53]:
@code_native debuginfo=:none  k(Int64, 10)

	.text
	testq	%rdi, %rdi
	jle	L38
	movq	%rdi, %rax
	sarq	$63, %rax
	andnq	%rdi, %rax, %rax
	movq	%rax, %rcx
	nopw	%cs:(%rax,%rax)
L32:
	decq	%rcx
	jne	L32
	retq
L38:
	xorl	%eax, %eax
	retq
	nopl	(%rax)


In [49]:
function m(::Type{T}, N) where T
    acc = zero(T)
    for i in 1:N
        acc += one(T)
        escape(acc)
    end
    return acc
end

m (generic function with 1 method)

In [54]:
@code_llvm debuginfo=:none m(Int64, 30)

define i64 @julia_m_4634(i64 signext %0) #0 {
top:
  %.inv = icmp sgt i64 %0, 0
  %1 = select i1 %.inv, i64 %0, i64 0
  br i1 %.inv, label %L12, label %L29

L12:                                              ; preds = %L12, %top
  %value_phi2 = phi i64 [ %3, %L12 ], [ 1, %top ]
  %value_phi3 = phi i64 [ %2, %L12 ], [ 0, %top ]
  %2 = add nuw nsw i64 %value_phi3, 1
  call void asm sideeffect "", "X,~{memory}"(i64 %2) #2
  %.not = icmp eq i64 %value_phi2, %1
  %3 = add nuw i64 %value_phi2, 1
  br i1 %.not, label %L29, label %L12

L29:                                              ; preds = %L12, %top
  %value_phi6 = phi i64 [ 0, %top ], [ %2, %L12 ]
  ret i64 %value_phi6
}


In [36]:
result2 = @benchmark m($Int64, $N)

BenchmarkTools.Trial: 199 samples with 1 evaluation.
 Range (min … max):  24.708 ms … 46.069 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     25.011 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.232 ms ±  1.643 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃ █▄▃                                                        
  █████▅▆▄▂▁▄▃▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  24.7 ms         Histogram: frequency by time        30.3 ms <

 Memory estimate: 32 bytes, allocs estimate: 2.

In [37]:
@benchmark m($Int64, $(N*10))

BenchmarkTools.Trial: 20 samples with 1 evaluation.
 Range (min … max):  248.565 ms … 261.804 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     249.763 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   251.196 ms ±   3.736 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ▁  ▁▁▁                                                       
  █▆█▆▆███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▆▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▆ ▁
  249 ms           Histogram: frequency by time          262 ms <

 Memory estimate: 32 bytes, allocs estimate: 2.

In [38]:
t = time(minimum(result2)) * u"ns" # in ns
pFreq = N/t |> u"GHz"

4.047241671242073 GHz

Pfff, this makes sense, 4 GHz is much closer to the frequency of my actual processor 

Note: Benchmarking is hard, careful evalutaion of *what* you are trying to benchmark.

- If we were just interesting in how fast `f(N)` was we would have been fine with our first measurement
- But we were interested in the speed of addition as a proxy of perfromance
- Integer math on a computer is associative, Floating-Point math is not.

## Coming back to `h`
    
```julia
function h(N)
    acc = 0.0
    for i in 1:N
        acc += 1.0
    end
    acc
end
```

In [55]:
@benchmark h($N)

BenchmarkTools.Trial: 68 samples with 1 evaluation.
 Range (min … max):  74.130 ms … 74.243 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     74.177 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   74.179 ms ± 28.005 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▃ █  ▃▃ ▃ █ ▃ █ ▃▃    ▃  █  ██ ▃ ▃▃  ▃      █        
  ▇▇▇▇▇▇▇▁█▁█▁▇██▇█▇█▁█▇█▁██▁▇▁▇█▇▇█▁▁██▇█▇██▇▁█▁▇▁▁▁▇█▇▁▁▁▁▇ ▁
  74.1 ms         Histogram: frequency by time        74.2 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [56]:
function l(N)
    acc = 0.0
    @simd for i in 1:N
        acc += 1.0
    end
    acc
end

l (generic function with 1 method)

In [57]:
@benchmark l($(N))

BenchmarkTools.Trial: 1069 samples with 1 evaluation.
 Range (min … max):  4.631 ms …  5.352 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.642 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.674 ms ± 64.570 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▅▄▃▁    ▁▂▃▂▁▁ ▁        ▁ ▁                               
  ███████▅▇██████████▇████▇██▆███▆█▇▇▆▄▅▆▅▄▅▆▁▅▅▄▁▄▅▅▆▅▆▅▅▅▅ █
  4.63 ms      Histogram: log(frequency) by time     4.87 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

# Performance annotiations in Julia

- https://docs.julialang.org/en/v1/manual/performance-tips/
- Julia does bounds checking by default `ones(10)[11]` is an error
- `@inbounds` Turns of bounds-checking locally
- `@fastmath` Turns of strict IEEE749 locally -- be very careful this might not to what you want
- `@simd` and `@simd ivdep` stronger gurantuees to encourage LLVM to use SIMD operations

In [39]:
?@simd

```
@simd
```

Annotate a `for` loop to allow the compiler to take extra liberties to allow loop re-ordering

!!! warning
    This feature is experimental and could change or disappear in future versions of Julia. Incorrect use of the `@simd` macro may cause unexpected results.


The object iterated over in a `@simd for` loop should be a one-dimensional range. By using `@simd`, you are asserting several properties of the loop:

  * It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
  * Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In many cases, Julia is able to automatically vectorize inner for loops without the use of `@simd`. Using `@simd` gives the compiler a little extra leeway to make it possible in more situations. In either case, your inner loop should have the following properties to allow vectorization:

  * The loop must be an innermost loop
  * The loop body must be straight-line code. Therefore, [`@inbounds`](@ref) is   currently needed for all array accesses. The compiler can sometimes turn   short `&&`, `||`, and `?:` expressions into straight-line code if it is safe   to evaluate all operands unconditionally. Consider using the [`ifelse`](@ref)   function instead of `?:` in the loop if it is safe to do so.
  * Accesses must have a stride pattern and cannot be "gathers" (random-index   reads) or "scatters" (random-index writes).
  * The stride should be unit stride.

!!! note
    The `@simd` does not assert by default that the loop is completely free of loop-carried memory dependencies, which is an assumption that can easily be violated in generic code. If you are writing non-generic code, you can use `@simd ivdep for ... end` to also assert that:


  * There exists no loop-carried memory dependencies
  * No iteration ever waits on a previous iteration to make forward progress.


In [59]:
@code_llvm debuginfo=:none l(10)

define double @julia_l_4677(i64 signext %0) #0 {
top:
  %.inv = icmp sgt i64 %0, 0
  %1 = select i1 %.inv, i64 %0, i64 0
  br i1 %.inv, label %L14.preheader, label %L38

L14.preheader:                                    ; preds = %top
  %min.iters.check = icmp ult i64 %1, 16
  br i1 %min.iters.check, label %L14, label %vector.ph

vector.ph:                                        ; preds = %L14.preheader
  %n.vec = and i64 %1, 9223372036854775792
  br label %vector.body

vector.body:                                      ; preds = %vector.body, %vector.ph
  %index = phi i64 [ 0, %vector.ph ], [ %index.next, %vector.body ]
  %vec.phi = phi <4 x double> [ zeroinitializer, %vector.ph ], [ %2, %vector.body ]
  %vec.phi7 = phi <4 x double> [ zeroinitializer, %vector.ph ], [ %3, %vector.body ]
  %vec.phi8 = phi <4 x double> [ zeroinitializer, %vector.ph ], [ %4, %vector.body ]
  %vec.phi9 = phi <4 x double> [ zeroinitializer, %vector.ph ], [ %5, %vector.body ]
  %2 = fadd fast <4 x double> %ve

# Let's revisit our example from earlier!

Slightly more complicated function!

- What is wrong with `mysum3(ones(10_000))`

In [61]:
function mysum3(data::Vector{T}) where T<:Number
  acc = zero(T)
  @simd for x in data
      acc += x
  end
  return acc
end

mysum3 (generic function with 1 method)

In [62]:
@code_llvm mysum3(zeros(3))

;  @ In[61]:1 within `mysum3`
define double @julia_mysum3_4706({}* nonnull align 16 dereferenceable(40) %0) #0 {
top:
;  @ In[61]:3 within `mysum3`
; ┌ @ simdloop.jl:71 within `macro expansion`
; │┌ @ simdloop.jl:51 within `simd_inner_length`
; ││┌ @ array.jl:215 within `length`
     %1 = bitcast {}* %0 to { i8*, i64, i16, i16, i32 }*
     %2 = getelementptr inbounds { i8*, i64, i16, i16, i32 }, { i8*, i64, i16, i16, i32 }* %1, i64 0, i32 1
     %3 = load i64, i64* %2, align 8
; └└└
; ┌ @ simdloop.jl:72 within `macro expansion`
; │┌ @ int.jl:83 within `<`
    %.not = icmp eq i64 %3, 0
; │└
   br i1 %.not, label %L18, label %L10.lr.ph

L10.lr.ph:                                        ; preds = %top
   %4 = bitcast {}* %0 to double**
   %5 = load double*, double** %4, align 8
; └
; ┌ @ simdloop.jl:75 within `macro expansion`
   %min.iters.check = icmp ult i64 %3, 16
   br i1 %min.iters.check, label %L10, label %vector.ph

vector.ph:                                        ; preds = %L10.

    %exitcond.not = icmp eq i64 %23, %3
; │└
   br i1 %exitcond.not, label %L18, label %L10

L18:                                              ; preds = %L10, %middle.block, %top
   %value_phi2 = phi double [ 0.000000e+00, %top ], [ %19, %middle.block ], [ %22, %L10 ]
; └
;  @ In[61]:6 within `mysum3`
  ret double %value_phi2
}
